In [1]:
import numpy as np
from qutip import basis, ket2dm, qeye

In [ ]:
from qutip import qeye

def depolarize_manual(rho, p):
    d = rho.shape[0]
    identity = qeye(d)
    return (1 - p) * rho + p * (identity / d)


In [ ]:
import numpy as np
from qutip import basis, ket2dm, qeye

def b92_simulation_with_qber(n_bits=1000, theta_deg=25, test_fraction=0.2):
    np.random.seed(42)

    # Define states
    theta = np.deg2rad(theta_deg)
    u0 = basis(2, 0)
    u1 = np.cos(theta) * basis(2, 0) + np.sin(theta) * basis(2, 1)
    rho_u0 = ket2dm(u0)
    rho_u1 = ket2dm(u1)
    I = qeye(2)
    P0 = I - rho_u1
    P1 = I - rho_u0

    alice_bits = np.random.randint(0, 2, n_bits)
    bob_measurements = np.random.randint(0, 2, n_bits)

    final_key = []
    alice_kept = []
    noise_probability = 0.5  # e.g. 5% depolarizing noise

    for i in range(n_bits):
        # Prepare clean state
        state = rho_u0 if alice_bits[i] == 0 else rho_u1
        
        # Apply depolarizing noise
        noisy_state = depolarize_manual(state, p=noise_probability)

        # Proceed with Bob's measurement
        if bob_measurements[i] == 0:
            p_pos = (P0 * noisy_state).tr().real
        else:
            p_pos = (P1 * noisy_state).tr().real

        outcome = np.random.choice(["+", "-"], p=[p_pos, 1 - p_pos])
        if outcome == "+":
            final_key.append(alice_bits[i])
            alice_kept.append(alice_bits[i])


    # QBER calculation: Compare a fraction of bits
    final_key = np.array(final_key)
    sample_size = int(test_fraction * len(final_key))
    
    if sample_size == 0 or len(final_key) < 2:
        return final_key.tolist(), None  # Not enough bits to test QBER

    indices = np.random.choice(len(final_key), size=sample_size, replace=False)
    alice_sample = final_key[indices]
    bob_sample = final_key[indices]  # In this simulation, Bob's guess is assumed correct unless noise introduced
    qber = np.sum(alice_sample != bob_sample) / sample_size

    # Remove test bits from final key (to simulate realistic scenario)
    mask = np.ones(len(final_key), dtype=bool)
    mask[indices] = False
    final_key_remaining = final_key[mask]

    return final_key_remaining.tolist(), qber

# Example usage
if __name__ == "__main__":
    key, qber = b92_simulation_with_qber(n_bits=127, theta_deg=25, test_fraction=0.2)
    print("Length of final key (after QBER check):", len(key))
    print("Sample final key bits:", key[:20])
    if qber is not None:
        print(f"Quantum Bit Error Rate (QBER): {qber*100:.2f}%")
    else:
        print("Not enough bits to compute QBER.")


Length of final key (after QBER check): 34
Sample final key bits: [1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0]
Quantum Bit Error Rate (QBER): 0.00%
